In [6]:
#pip install pyyaml h5py  # Required to save models in HDF5 format

In [7]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.9.2


In [8]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [9]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [10]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
22/32 [===================>..........] - ETA: 0s - loss: 1.3956 - sparse_categorical_accuracy: 0.6065  
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 3s 14ms/step - loss: 1.2061 - sparse_categorical_accuracy: 0.6610 - val_loss: 0.7163 - val_sparse_categorical_accuracy: 0.7920
Epoch 2/10
23/32 [====================>.........] - ETA: 0s - loss: 0.4286 - sparse_categorical_accuracy: 0.8832
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.4282 - sparse_categorical_accuracy: 0.8810 - val_loss: 0.5266 - val_sparse_categorical_accuracy: 0.8430
Epoch 3/10
23/32 [====================>.........] - ETA: 0s - loss: 0.2922 - sparse_categorical_accuracy: 0.9226
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 0.2870 - sparse_categorical_accuracy: 0.9270 - val_loss: 0.4831 - val_sparse_categorical_accuracy: 0.8470
Epoch 4/10
23/32 [=======

In [11]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.index', 'cp.ckpt.data-00000-of-00001']

In [12]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3426 - sparse_categorical_accuracy: 0.0710 - 132ms/epoch - 4ms/step
Untrained model, accuracy:  7.10%


In [13]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4162 - sparse_categorical_accuracy: 0.8630 - 65ms/epoch - 2ms/step
Restored model, accuracy: 86.30%


In [14]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [15]:
os.listdir(checkpoint_dir)

['cp-0025.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001']

In [16]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [17]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4824 - sparse_categorical_accuracy: 0.8800 - 138ms/epoch - 4ms/step
Restored model, accuracy: 88.00%


In [18]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4824 - sparse_categorical_accuracy: 0.8800 - 143ms/epoch - 4ms/step
Restored model, accuracy: 88.00%


In [19]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1570 - sparse_categorical_accuracy: 0.6660
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4425 - sparse_categorical_accuracy: 0.8850
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2749 - sparse_categorical_accuracy: 0.9290
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2119 - sparse_categorical_accuracy: 0.9580
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1564 - sparse_categorical_accuracy: 0.9680


In [20]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets	keras_metadata.pb  saved_model.pb  variables


In [21]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4274 - sparse_categorical_accuracy: 0.8660 - 148ms/epoch - 5ms/step
Restored model, accuracy: 86.60%
32/32 [==============================] - 0s 1ms/step
(1000, 10)


In [23]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1716 - sparse_categorical_accuracy: 0.6620
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4267 - sparse_categorical_accuracy: 0.8840
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2855 - sparse_categorical_accuracy: 0.9300
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1984 - sparse_categorical_accuracy: 0.9560
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1511 - sparse_categorical_accuracy: 0.9700


In [24]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))



32/32 - 0s - loss: 0.4222 - sparse_categorical_accuracy: 0.8620 - 135ms/epoch - 4ms/step
Restored model, accuracy: 86.20%
